# Chapter 7: Data Wrangling (Clean, Transform, Merge, Reshape)
> Much of the programming work in data analysis and modeling is spent on data preparation: loading, cleaning, transforming, and rearranging. Sometimes the way that data
is stored in files or databases is not the way you need it for a data processing application.
Many people choose to do ad hoc processing of data from one form to another using
a general purpose programming, like Python, Perl, R, or Java, or UNIX text processing
tools like sed or awk. Fortunately, pandas along with the Python standard library pro-
vide you with a high-level, flexible, and high-performance set of core manipulations
and algorithms to enable you to wrangle data into the right form without much trouble.
If you identify a type of data manipulation that isn’t anywhere in this book or elsewhere
in the pandas library, feel free to suggest it on the mailing list or GitHub site. Indeed,
much of the design and implementation of pandas has been driven by the needs of real
world applications.

**Overview**:
* Combining and Merging Data Sets
* Reshaping and Pivoting
* Data Transformation
* String Manipulation
* Example: USDA Food Database

# Combining and Merging Data Sets

* **pandas.merge**: connects rows in DataFrames based on one or more keys. This will
be familiar to users of SQL or other relational databases, as it implements database
join operations.
* **pandas.concat** glues or stacks together objects along an axis.
* **combine_first** instance method enables splicing together overlapping data to fill
in missing values in one object with values from another.

## Database-style DataFrame Merges

In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

### **Case 1**: Merge many-to-one: One DataFrame has multiple rows, and one has one row for each value

In [2]:
df1 = DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    'data1': np.arange(7)
    })
df2 = DataFrame({
        'key': ['a', 'b', 'd'], 
        'data2': np.arange(3)
    })

**pd.merge** is able to merge if both DataFrame has at least 1 common columns, it will keep the same value

In [4]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


We can specify which column to jon in by using **on** option

In [5]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


If the column names are different in each object, you can specify them separately by using **left_on** and **right_on** options:

In [6]:
df3 = DataFrame({
        'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
        'data1': np.arange(7)
    })
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [7]:
df4 = DataFrame({
        'rkey': ['a', 'b', 'd'], 
        'data2': np.arange(3)
    })
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [8]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


**pd.merge** by default  does an **'inner' join**, then some rows of each DataFrame can be lost. 

If we don't want to lose any data, then we can change option **how** of **pd.merge** from **inner** by default to **outer**

In [9]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


### Case 2: Merge many-to-many
Many-to-many merges have well-defined though not necessarily intuitive behavior.

In [10]:
df1 = DataFrame({
        'key': ['b', 'b', 'a', 'c', 'a', 'b'],
        'data1': range(6)
    })
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [11]:
df2 = DataFrame({
        'key': ['a', 'b', 'a', 'b', 'd'],
        'data2': range(5)
    })
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


**how='left'**: Keep all data of df1

In [12]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [13]:
pd.merge(df1, df2, on='key', how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


Merge with multiple keys:

In [14]:
left = DataFrame({
        'key1': ['foo', 'foo', 'bar'],
        'key2': ['one', 'two', 'one'],
        'lval': [1, 2, 3]
    })
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [15]:
right = DataFrame({
        'key1': ['foo', 'foo', 'bar', 'bar'],
        'key2': ['one', 'one', 'one', 'two'],
        'rval': [4, 5, 6, 7]
    })
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [16]:
pd.merge(left, right, on =['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


To determine which key combinations will appear in the result depending on the choice
of merge method, think of the multiple keys as forming an array of tuples to be used
as a single join key (even though it’s not actually implemented that way).

A last issue to consider in merge operations is the treatment of overlapping column
names. While you can address the overlap manually (see the later section on renaming
axis labels), merge has a suffixes option for specifying strings to append to overlapping
names in the left and right DataFrame objects:

In [17]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [18]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## Merging on Index

In some cases, the merge key or keys in a DataFrame will be found in its index. In this
case, you can pass left_index=True or right_index=True (or both) to indicate that the
index should be used as the merge key:

In [5]:
left1 = DataFrame({
    'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    'value': range(6)
})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [8]:
right1 = DataFrame(
        {'group_val': [3.5, 7]}, 
        index=['a', 'b']
    )
right1

,group_val
a,3.5
b,7.0


In [9]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Since the default merge method is to intersect the join keys, you can instead form the
union of them with an outer join:

In [10]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


With herachical index, it is a bit more complicated:

In [12]:
lefth = DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)
})
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [14]:
righth = DataFrame(
    np.arange(12).reshape((6,2)), 
    index=[
        ['Nevada', 'Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio'],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ],
    columns=['event1', 'event2']
)
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
       2000       4       5
Ohio   2000       6       7
       2001       8       9
       2002      10      11

In [15]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


Using the indexes of both sides of the merge is also not an issue:

In [16]:
left2 = DataFrame(
    [[1., 2.], [3., 4.], [5., 6.]], 
    index=['a', 'c', 'e'],
    columns=['Ohio', 'Nevada']
)
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [17]:
right2 = DataFrame(
    [[7., 8.], [9., 10.], [11., 12.], [13, 14]],
    index=['b', 'c', 'd', 'e'], 
    columns=['Missouri', 'Alabama']
)
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [18]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as
an alternative to using the more general **concat** function described below:

In [19]:
another = DataFrame(
    [[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
    index=['a', 'c', 'e', 'f'], 
    columns=['New York', 'Oregon']
)
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [21]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


## Concatenating Along an Axis

Another kind of data combination operation is alternatively referred to as concatenation, binding, or stacking. NumPy has a concatenate function for doing this with raw
**NumPy** arrays:

In [23]:
arr = np.arange(10).reshape(5,2)
arr

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [25]:
np.concatenate([arr, arr], axis=1)

array([[0, 1, 0, 1],
       [2, 3, 2, 3],
       [4, 5, 4, 5],
       [6, 7, 6, 7],
       [8, 9, 8, 9]])